In [5]:
import tweepy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import json
from config_twitter import consumer_key, consumer_secret, access_token, access_token_secret
import time
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,parser=tweepy.parsers.JSONParser())

In [6]:
consumer_key

'DXO0bbdDlwl6R0ruG1MZWvbxS'

In [7]:
file_one = pd.read_csv("house_data.csv")
file_one = file_one.drop(columns = ["pure_urban.1","rural_suburban_mix","sparse_suburban","urban_suburban_mix","flipped_seat.1","Unnamed: 44","Unnamed: 45","Unnamed: 46","Unnamed: 47","Unnamed: 48","dense_suburban","pure_urban","Type of District","Unnamed: 35","Final?"])
file_one ["CD"] = file_one["State Abbreviation"] + "-" + file_one["CD#"].map(str)
file_one ["Last Two Pres Winners"] = file_one["2012 Winner"] + "/" + file_one["2016 winner"]
file_one = file_one.set_index('CD')
file_one["Dem %"] = file_one["Dem %"].str.replace('%', '')
file_one["GOP %"] = file_one["GOP %"].str.replace('%', '')
file_one["Other %"] = file_one["Other %"].str.replace('%', '')
file_one["Dem Margin"] = file_one["Dem Margin"].str.replace('%', '')
file_one["2016 Clinton Margin"] = file_one["2016 Clinton Margin"].str.replace('%', '')
file_one["Swing vs. 2016 Prez"] = file_one["Swing vs. 2016 Prez"].str.replace('%', '')
file_one.head()







,Democrat Strength Rating,State,State Abbreviation,CD#,2018 Cook PVI Score,2018 Winner,Party,flipped_seat,Dem Votes,GOP Votes,...,Trump Vote Share,Obama 2012,Romney 2012,Obama 2008,McCain 2008,2016-2012,2016-2008,2012-2008,City Lab Congressional Index,Last Two Pres Winners
CD,,,,,,,,,,,,,,,,,,,,,
FL-25,337,Florida,FL,25,-4,Mario Diaz-Balart,R,NaN,"84,173","128,672",...,49.7,44.9,54.5,40.6,58.8,3.0,7.3,4.3,Urban-suburban mix,Romney/Trump
UT-3,412,Utah,UT,3,-25,John Curtis,R,NaN,"70,686","174,856",...,47.2,19.5,78.3,29.5,67.5,3.8,6.2,10.0,Sparse suburban,Romney/Trump
CA-21,234,California,CA,21,5,T.J. Cox,D,1.0,"57,239","56,377",...,39.7,54.6,43.5,52.0,46.0,0.6,3.2,2.6,Rural-suburban mix,Obama /Clinton
FL-26,228,Florida,FL,26,6,Debbie Mucarsel-Powell,D,1.0,"119,797","115,678",...,40.5,55.4,43.9,51.5,47.9,1.4,5.3,3.9,Urban-suburban mix,Obama /Clinton
FL-27,210,Florida,FL,27,5,Donna Shalala,D,1.0,"130,743","115,588",...,38.9,53.0,46.3,50.1,49.3,5.5,8.4,2.9,Urban-suburban mix,Obama /Clinton


In [25]:
house_one = file_one.iloc[:,0:18]
house_two = file_one.iloc[:,19:35]

In [26]:
house_two

,2012 Winner,2016 winner,Pre-Election Rating,PVI Performance,Hold/ Flip,Clinton Vote Share,Trump Vote Share,Obama 2012,Romney 2012,Obama 2008,McCain 2008,2016-2012,2016-2008,2012-2008,City Lab Congressional Index,Last Two Pres Winners
CD,,,,,,,,,,,,,,,,
FL-25,Romney,Trump,Likely R,16.9,R Hold,47.9,49.7,44.9,54.5,40.6,58.8,3.0,7.3,4.3,Urban-suburban mix,Romney/Trump
UT-3,Romney,Trump,Solid R,15.2,R Hold,23.3,47.2,19.5,78.3,29.5,67.5,3.8,6.2,10.0,Sparse suburban,Romney/Trump
CA-21,Obama,Clinton,Likely R,4.2,D Flip,55.2,39.7,54.6,43.5,52.0,46.0,0.6,3.2,2.6,Rural-suburban mix,Obama /Clinton
FL-26,Obama,Clinton,Toss Up,4.3,D Flip,56.8,40.5,55.4,43.9,51.5,47.9,1.4,5.3,3.9,Urban-suburban mix,Obama /Clinton
FL-27,Obama,Clinton,Likely D,1.0,D Flip,58.5,38.9,53.0,46.3,50.1,49.3,5.5,8.4,2.9,Urban-suburban mix,Obama /Clinton
WI-8,Romney,Trump,Solid R,20.4,R Hold,38.6,56.2,47.6,51.3,53.7,45.0,9.0,15.1,6.1,Rural-suburban mix,Romney/Trump
UT-1,Romney,Trump,Solid R,10.7,R Hold,22.4,49.7,20.4,77.4,29.4,67.7,2.0,7.0,9.0,Sparse suburban,Romney/Trump
NY-24,Obama,Clinton,Likely R,8.3,R Hold,48.9,45.3,57.0,41.1,56.0,42.0,8.1,7.1,1.0,Rural-suburban mix,Obama /Clinton
LA-5,Romney,Trump,Solid R,21.6,R Hold,34.1,63.5,37.7,61.0,36.7,62.0,3.6,2.6,1.0,Pure rural,Romney/Trump


In [27]:
house_one["Dem Votes"] = house_one["Dem Votes"].str.replace(',', '')
house_one['Dem Votes'] = house_one['Dem Votes'].astype(int)
house_one["GOP Votes"] = house_one["GOP Votes"].str.replace(',', '')
house_one['GOP Votes'] = house_one['GOP Votes'].astype(int)
house_one["Other Votes"] = house_one["Other Votes"].str.replace(',', '')
house_one['Other Votes'] = house_one['Other Votes'].astype(int)
house_one["2016"] = house_one["Other Votes"].str.replace(',', '')
house_one['Other Votes'] = house_one['Other Votes'].astype(int)
house_one ["2018"]
house_one.head()

,Democrat Strength Rating,State,State Abbreviation,CD#,2018 Cook PVI Score,2018 Winner,Party,flipped_seat,Dem Votes,GOP Votes,Other Votes,Dem %,GOP %,Other %,Dem Margin,2016 Clinton Margin,Swing vs. 2016 Prez,2016 Total Votes Cast
CD,,,,,,,,,,,,,,,,,,
FL-25,337,Florida,FL,25,-4,Mario Diaz-Balart,R,NaN,84173,128672,0,39.50,60.50,0.00,-20.90,-1.90,-19.00,"263,235"
UT-3,412,Utah,UT,3,-25,John Curtis,R,NaN,70686,174856,13316,27.30,67.50,5.10,-40.20,-23.90,-16.30,"289,925"
CA-21,234,California,CA,21,5,T.J. Cox,D,1.0,57239,56377,0,50.40,49.60,0.00,0.80,15.40,-14.60,"134,881"
FL-26,228,Florida,FL,26,6,Debbie Mucarsel-Powell,D,1.0,119797,115678,0,50.90,49.10,0.00,1.70,16.30,-14.60,"289,209"
FL-27,210,Florida,FL,27,5,Donna Shalala,D,1.0,130743,115588,6255,51.80,45.80,2.50,6.00,19.70,-13.70,"299,037"


In [28]:
pop_state = house_one.groupby("State").sum()
pop_state ["Dem House Vote Pct"] = pop_state


ValueError: Wrong number of items passed 7, placement implies 1